In [3]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, Pool

# === 1. Trainingsdaten laden & vorbereiten ===
df = pd.read_csv("train.csv")
X = df.drop(columns=["Id", "SalePrice"])
y = df["SalePrice"]

# Fehlende Werte behandeln
for col in X.columns:
    if X[col].dtype == "object":
        X[col].fillna("Missing", inplace=True)
    else:
        X[col].fillna(X[col].median(), inplace=True)

cat_features = X.select_dtypes(include="object").columns.tolist()

# === 2. CatBoost-Modell trainieren ===
model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.05,
    depth=6,
    loss_function='RMSE',
    random_seed=42,
    verbose=0  # Warnungen und Logs unterdrücken
)

train_pool = Pool(X, y, cat_features=cat_features)
model.fit(train_pool)

# === 3. Beispielhafte Benutzereingabe (manuell oder via Interface) ===

input_data = {
    "OverallQual": 7,
    "GrLivArea": 2000,
    "GarageCars": 2,
    "TotalBsmtSF": 1000,
    "FullBath": 2,
    "YearBuilt": 2005,
    "Neighborhood": "CollgCr",
    "HouseStyle": "2Story",
    "ExterQual": "Gd",
    "KitchenQual": "Gd",
    "GarageFinish": "Unf",
}

# === 4. Eingabedaten als DataFrame vorbereiten ===
input_df = pd.DataFrame([input_data])

# Alle fehlenden Spalten (nicht eingegeben) ergänzen mit Median/"Missing"
for col in X.columns:
    if col not in input_df.columns:
        if col in cat_features:
            input_df[col] = "Missing"
        else:
            input_df[col] = X[col].median()

# Spaltenreihenfolge anpassen
input_df = input_df[X.columns]

# === 5. Vorhersage durchführen ===
input_pool = Pool(input_df, cat_features=cat_features)
prediction = model.predict(input_pool)

print(f"🏠 Geschätzter Hauspreis: ${prediction[0]:,.2f}")


🏠 Geschätzter Hauspreis: $190,049.81
